In [4]:
import paramiko
import time
import threading
import argparse

min_port = 20101
max_port = 20101
client = 1
experiment = 1 

class JetsonSSHClient:
  def __init__(self, port):
    self.port = port
    self.cli = paramiko.SSHClient()
    self.cli.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    self.cli.connect("147.47.200.209", port=port, username="jetson", password="jetson")
    self.channel = self.cli.invoke_shell()
    
  def execute(self, command = "ls -al"):
    command = command +"\n"
    self.channel.send(command)
    # 결과 수신
    
    time.sleep(3)
    output = self.channel.recv(65535).decode("utf-8")

    if self.port == 20101:
      print(output.strip(command))
    #print(output.split("jetson@jetson-desktop:~$")[0])
    #print(output.split("jetson@jetson-desktop:~$")[1])

  def close(self):
    self.cli.close()

class JetsonController:
  def __init__(self, min_port = 20101, max_port = 20101):
    self.clients = {}
    self.threads = {}
    for port in range(min_port, max_port + 1):
      self.clients[port] = JetsonSSHClient(port)
      self.threads[port] = None

  def global_execute(self, command = "ls -al"):
    for port, client in self.clients.items():
      thread = threading.Thread(target = client.execute, args=(command,))
      self.threads[port] = thread
      thread.start()

  def check_status(self):
    for thread in self.threads.values():
      if thread.is_alive() == True:
        return False
    return True

In [10]:
import requests
import json
reset = requests.get("http://147.47.200.178:9103/reset")

client_num_in_json = json.dumps(client)
res = requests.put("http://147.47.200.178:9103/client_num/", data=client_num_in_json)
exp_in_json = json.dumps(experiment)
res = requests.put("http://147.47.200.178:9103/experiment/", data=exp_in_json)

In [6]:
jetsonController = JetsonController(min_port=min_port, max_port=max_port)
#command = "docker start fl && docker attach fl && cd /ambient_fl/FL_Client/ && python3 jetson_client.py --ip 147.47.200.178 --p 9103"
#jetsonController.global_execute(command)

In [7]:
while True:
    command = input()
    jetsonController.global_execute(str(command))

ls
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 4.9.201-tegra aarch64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

70 updates can be applied immediately.
35 of these updates are standard security updates.
To see these additional updates run: apt list --upgradable

Last login: Fri Sep 24 11:04:37 2021 from 147.47.7.75
jetson@jetson-desktop:~$ ls
ambient_fl  Downloads         jetson             Pictures   Videos
Desktop     examples.desktop  jetson_network.sh  Public
Documents   FL                Music              Templates
jetson@jetson-desktop:~$ 
docker attach fl

ls

exit
close

close
^CTraceback (most recent call last):
  File "jetson_client.py", line 420, in <module>
    client.task()
  File "jetson_

KeyboardInterrupt: Interrupted by user

In [9]:
#Ctrl+p, Ctrl+q 이후
your_id = 0
your_data_path = f"./id_{your_id}.json"
!docker cp docker_server:/home/ambient_fl/FL_Server/training_history.json your_data_path
import json
data = json.load(open(your_data_path, "r"))

'docker'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rounds = max(accuracies[0].keys())
ids = max(accuracies.keys())
result = np.zeros((ids, rounds))

for round_ in rounds:
    for cur_id in range(ids):
        result[cur_id, round_] = accuracies[round_][cur_id]
        
def get_result(id, round):
    return result[id, round]

def plot_accuracy():
    rounds = list(range(len(result[0])))
    rounds = rounds.astype("int8")
    for fed_id in range(len(result)):
        accs = result[fed_id, :]
        plt.plot(rounds, accs, label=f"{fed_id}")
        plt.xticks(range(len(result[0]+1)))
    plt.title("Accuracies of the clients (%)")
    plt.xlabel("rounds")
    plt.ylabel("accuracy (%)")
    plt.legend()
    plt.show() 
    
plot_accuracy()